In [1]:
import os
import sys
import glob
import pickle
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib.pyplot import imshow
from PIL import Image
% matplotlib inline

# Import modules every time you run code imported using %aimport
%load_ext autoreload
%autoreload 1

# Add the src directory for functions
src_dir = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'src')
print(src_dir)
sys.path.append(src_dir)

# import my functions:
%aimport models
from models import*
%aimport functions
from functions import*

# Base Directory where data is stored
base_data_dir = '/home/rbbidart/project/rbbidart/breakHis/'

/home/rbbidart/breakHis/src


Using TensorFlow backend.


In [11]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [2]:
size = 100
fold = 'fold1'
new_dir = 'train'

cur_loc = os.path.join(base_data_dir, 'mkfold_keras_8', fold, str(size), new_dir)
out_loc = os.path.join(base_data_dir, 'features', '', fold, str(size))
if not os.path.exists(out_loc):
    os.makedirs(out_loc)
    
model = load_model('/home/rbbidart/breakHis_out/inception_pre_249/fine_tune_.19-0.75.hdf5')

In [3]:
features_names, all_features = get_freatures_incp3(model, cur_loc, classes=8, image_shape = (512, 512))
print(features_names)
np.save(os.path.join(out_loc, new_dir+'_feat_incp3_'+str(size)+'_noaug.npy'), all_features)
with open(os.path.join(out_loc, new_dir+'_names_feat_incp3_'+str(size)+'_noaug.npy'), 'wb') as fp:
    pickle.dump(features_names, fp)

['SOB_M_LC-14-12204-100-048.png', 'SOB_M_LC-14-15570C-100-011.png', 'SOB_M_LC-14-16196-100-017.png', 'SOB_M_LC-14-15570C-100-010.png', 'SOB_M_LC-14-12204-100-042.png', 'SOB_M_LC-14-12204-100-058.png', 'SOB_M_LC-14-15570C-100-027.png', 'SOB_M_LC-14-15570C-100-033.png', 'SOB_M_LC-14-12204-100-039.png', 'SOB_M_LC-14-12204-100-061.png', 'SOB_M_LC-14-15570C-100-023.png', 'SOB_M_LC-14-15570C-100-001.png', 'SOB_M_LC-14-15570C-100-017.png', 'SOB_M_LC-14-15570C-100-005.png', 'SOB_M_LC-14-12204-100-033.png', 'SOB_M_LC-14-15570C-100-008.png', 'SOB_M_LC-14-12204-100-035.png', 'SOB_M_LC-14-16196-100-005.png', 'SOB_M_LC-14-12204-100-037.png', 'SOB_M_LC-14-15570C-100-004.png', 'SOB_M_LC-14-15570C-100-030.png', 'SOB_M_LC-14-16196-100-001.png', 'SOB_M_LC-14-15570C-100-018.png', 'SOB_M_LC-14-15570C-100-006.png', 'SOB_M_LC-14-12204-100-055.png', 'SOB_M_LC-14-12204-100-051.png', 'SOB_M_LC-14-12204-100-036.png', 'SOB_M_LC-14-15570C-100-003.png', 'SOB_M_LC-14-16196-100-009.png', 'SOB_M_LC-14-12204-100-050.p

In [12]:
from scipy.spatial import distance
image_shape = (512, 512)

incp3_features = pd.DataFrame(np.load('/home/rbbidart/project/rbbidart/breakHis/features/fold1/100/train_feat_incp3_100_noaug.npy'))
# image_loc = '/home/rbbidart/project/rbbidart/breakHis/mkfold_keras_8/fold1/100/test/B_A/SOB_B_A-14-22549G-100-001.png'
image_loc = '/home/rbbidart/project/rbbidart/breakHis/mkfold_keras_8/fold1/100/train/M_MC/SOB_M_MC-14-19979-100-007.png'

features_names = np.asarray(pickle.load(open('/home/rbbidart/project/rbbidart/breakHis/features/fold1/100/train_names_feat_incp3_100_noaug.npy', "rb")))
image = Image.open(image_loc)
image = np.array(image.resize(image_shape))

model = load_model('/home/rbbidart/breakHis_out/inception_pre_249/fine_tune_.19-0.75.hdf5')
conv_model = Model(inputs=model.input, outputs=model.get_layer(index=311).output)
image_features = conv_model.predict(np.expand_dims(image, axis=0))

In [13]:
print(image_features.shape)
distances = incp3_features.apply(lambda row: distance.euclidean(row, image_features), axis=1)

# Create a new dataframe with distances.
distance_frame = pd.DataFrame(data={"dist": distances, "idx": features_names})
distance_frame.sort("dist", inplace=True)
display(distance_frame)

# Find the most similar player to lebron (the lowest distance to lebron is lebron, the second smallest is the most similar non-lebron player)
# second_smallest = distance_frame.iloc[1]["idx"]
# most_similar_to_lebron = nba.loc[int(second_smallest)]["player"]

# nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
# distances, indices = nbrs.kneighbors(X)

(1, 2048)


/home/rbbidart/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  


,dist,idx
984,0.000000,SOB_M_MC-14-19979-100-007.png
188,4.000475,SOB_B_F-14-14134E-100-006.png
780,4.031521,SOB_M_DC-14-14926-100-015.png
507,4.059205,SOB_M_DC-14-14015-100-024.png
83,4.248959,SOB_B_F-14-21998EF-100-017.png
183,4.271206,SOB_B_F-14-21998EF-100-005.png
371,4.325571,SOB_M_DC-14-15572-100-023.png
227,4.335051,SOB_B_F-14-21998EF-100-021.png
167,4.353339,SOB_B_F-14-21998EF-100-033.png
659,4.407906,SOB_M_DC-14-15792-100-001.png
